# Smooth fit to log-odds ratios
**objective** : The objective of this study is to observe the influence/association between death from maternal exposure to X-rays and childhood cancer for subjects partitioned into 120 age and birth-year strata.



## 1. Data

Description : 
- $r^0_i$ : is the number of deaths among unexposed subjects in stratum $i$, 
- $n^0_i$ : is the total number of unexposed subjects in stratum $i$, 
- $r^1_i$ : is the number among exposed subjects in stratum $i$, 
- $n^1_i$ : is the total number of exposed subjects in stratum $i$
- $\text{year}_i$ : is the stratum-specific birth year (relative to 1954).

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("data/data.csv")
df

,r0,n0,r1,n1,year
0,0,28,3,28,-10
1,2,21,5,21,-9
2,2,32,2,32,-9
3,1,35,7,35,-8
4,2,35,7,35,-8
...,...,...,...,...,...
115,2,51,5,51,8
116,4,42,7,42,8
117,6,46,6,46,9
118,4,54,3,54,9


# Model 

The proposed model [source](https://pj.freefaculty.org/Ubuntu/15.04/amd64/openbugs/openbugs-3.2.3/doc/Examples/Oxford.html) is as follows. $\forall i \text{ (stratum)} $

-   $r_i^0 \sim \text{Binomial}(n_i^0,p_i^0)$
-   $r_i^1 \sim \text{Binomial}(n_i^1,p_i^1)$.

The parameters $p_i^0$, $p_i^1$ are given by :

-   $p_i^0 = \text{sigmoid}(\mu_i)$
-   $p_i^1 = \text{sigmoid}(\mu_i+log\psi_i)$
    -   $\log\psi_i = \alpha+\beta_1 year_i+\beta_2(year_i^2-22)+b_i $
    -   $p_i^1 = \text{sigmoid}(\mu_i+\alpha+\beta_1 year_i+\beta_2(year_i^2-22)+b_i)$ and $ b_i \sim\mathcal N(0,\tau)$


In [1]:
import numpy as np

def sigmoid(x):
    y = 1 / (1 + np.exp(-x))
    return y

def p0(mu):
    return sigmoid(mu)

def p1(mu, alpha, beta1, beta2, b, year):
    return sigmoid(mu + alpha + beta1*year + beta2*(year**2-22) + b)

def lvr0(mu, r0, n0):
    r = 0
    for i in range(len(mu)):
        r += r0[i] * np.log(p0(mu[i])) + (n0[i] - r0[i]) * np.log(1 - p0(mu[i]))
    return r

def lvr1(mu, alpha, beta1, beta2, b, year, r1, n1):
    r = 0
    for i in range(len(mu)):
        r += r1[i] * np.log(p1(mu[i], alpha, beta1, beta2, b[i], year[i])) + (n1[i] - r1[i]) * np.log(1 - p1(mu[i], alpha, beta1, beta2, b[i], year[i]))
    return r

In [2]:
def init():
    ni = df.shape[0]
    alpha = 0
    beta1 = 0
    beta2 = 0
    sigma = 1
    mu = np.zeros(100)
    b = np.zeros(100)
    init = np.concatenate((np.array([alpha, beta1, beta2, sigma]), mu, b))